<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/AI_agents/Router_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries

In [1]:
!pip install llama-index llama-index-llms-openai openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
OpenAI_API = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OpenAI_API

## Import Libraries

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Set the llm and embedding model to the LlamaIndex Settings
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

## Download and load Data

In [4]:
# Create directory to store pdfs in it
!mkdir data
%cd data

/content/data


In [5]:
!wget https://ocw.mit.edu/courses/1-264j-database-internet-and-systems-integration-technologies-fall-2013/d549b3ecf40310a93fec5da29a293fd5_MIT1_264JF13_lect_15.pdf
!wget https://web.mit.edu/~csvoss/Public/usabo/stats_handout.pdf

--2025-02-12 14:19:07--  https://ocw.mit.edu/courses/1-264j-database-internet-and-systems-integration-technologies-fall-2013/d549b3ecf40310a93fec5da29a293fd5_MIT1_264JF13_lect_15.pdf
Resolving ocw.mit.edu (ocw.mit.edu)... 151.101.130.133, 151.101.194.133, 151.101.66.133, ...
Connecting to ocw.mit.edu (ocw.mit.edu)|151.101.130.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160716 (157K) [application/pdf]
Saving to: ‘d549b3ecf40310a93fec5da29a293fd5_MIT1_264JF13_lect_15.pdf’

d549b3ecf40310a93fe 100%[===================>] 156.95K   977KB/s    in 0.2s    

2025-02-12 14:19:09 (977 KB/s) - ‘d549b3ecf40310a93fec5da29a293fd5_MIT1_264JF13_lect_15.pdf’ saved [160716/160716]

--2025-02-12 14:19:09--  https://web.mit.edu/~csvoss/Public/usabo/stats_handout.pdf
Resolving web.mit.edu (web.mit.edu)... 104.115.233.48, 2600:1417:76:480::255e, 2600:1417:76:4a0::255e
Connecting to web.mit.edu (web.mit.edu)|104.115.233.48|:443... connected.
HTTP request sent, awaiting respo

In [6]:
# Generate two vector stores, one for statistics and another for SQL.
def load_sql():
  documents = SimpleDirectoryReader(input_files=["/content/data/d549b3ecf40310a93fec5da29a293fd5_MIT1_264JF13_lect_15.pdf"]).load_data()
  splitter = SentenceSplitter(chunk_size=1024)
  nodes = splitter.get_nodes_from_documents(documents)
  vector_index = VectorStoreIndex(nodes)
  vector_query_engine = vector_index.as_query_engine()
  return vector_query_engine

def load_stats():
  documents = SimpleDirectoryReader(input_files=["/content/data/stats_handout.pdf"]).load_data()
  splitter = SentenceSplitter(chunk_size=1024)
  nodes = splitter.get_nodes_from_documents(documents)
  vector_index = VectorStoreIndex(nodes)
  vector_query_engine = vector_index.as_query_engine()
  return vector_query_engine

In [7]:
# load the vector indexes to query engine tool to make them available for retrieval with description for the
# router engine to make decision based on user query

sql_tool = QueryEngineTool.from_defaults(
    query_engine=load_sql(),
    description=(
        "Useful for retrieving specific context related to SQL from the SQL lecture."
    ),
)

stats_tool = QueryEngineTool.from_defaults(
    query_engine=load_stats(),
    description=(
        "Useful for retrieving specific context related to statistics from the stats handbook."
    ),
)

## Agentic Rag with Router

LLMSingleSelector: This is a selector that uses the LLM to select a single choice from a list of choices.

In [8]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Load the Router query engine module with llm and tools

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        sql_tool,
        stats_tool,
    ],
    verbose=True
)

In [9]:
response = query_engine.query("What are transactions?")
print(response)

Selecting query engine 0: Transactions are a concept related to SQL, and the SQL lecture would provide specific context about them..
Transactions are sequences of operations performed as a single logical unit of work in a database. They ensure that a series of actions either complete successfully or are entirely rolled back, maintaining the integrity of the database. Transactions are characterized by properties known as ACID: Atomicity, Consistency, Isolation, and Durability. These properties help manage issues that arise in multi-user databases, such as conflicting actions and ensuring that data remains accurate and reliable even in the event of failures.


In [10]:
response = query_engine.query("What is variance?")
print(response)

Selecting query engine 1: Variance is a statistical concept, and the stats handbook would provide relevant context and information about it..
Variance is a statistical measure that represents the degree of spread or dispersion of a set of values. It quantifies how much the values in a dataset differ from the mean (average) of that dataset. A higher variance indicates that the values are more spread out, while a lower variance suggests that they are closer to the mean. Variance is calculated by taking the average of the squared differences between each value and the mean.


In [11]:
query_engine.query("What is the skeness in stats?")

Selecting query engine 1: The question pertains to statistics, and choice 2 is specifically related to retrieving context from a stats handbook..


Response(response='Skewness in statistics refers to the measure of asymmetry in the distribution of data. It indicates whether the data points are concentrated on one side of the mean, which can help in understanding the shape of the distribution. A positive skewness suggests that the tail on the right side is longer or fatter than the left side, while a negative skewness indicates that the tail on the left side is longer or fatter than the right side. A skewness close to zero suggests a symmetrical distribution.', source_nodes=[NodeWithScore(node=TextNode(id_='bb0d7f56-e3d5-4534-ae57-ed7a78b2fe8b', embedding=None, metadata={'page_label': '7', 'file_name': 'stats_handout.pdf', 'file_path': '/content/data/stats_handout.pdf', 'file_type': 'application/pdf', 'file_size': 1367357, 'creation_date': '2025-02-12', 'last_modified_date': '2014-10-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_

To allow using multiple tools at once use LLMMultiSelector